In [ ]:
# Version of SciUnit is still the same, but the code has changed. Better to install a specific commit
# !pip install -q git+git://github.com/scidash/sciunit.git@9e61c060cd8e5c6d7bce675d37c398987a9491c5
# For sciunit 0.2.0.2
# !pip install quantities==0.12.1
# pip install neurom==1.14.6

# !pip install -q neo
# !pip install -q elephant
# !pip install -q gitpython
# !pip install -q efel

!pip install scipy seaborn
!pip install -U git+https://github.com/appukuttan-shailesh/hbp-validation-client.git
!pip install --quiet fpdf PyPDF2 tabulate
# First downgrade your pip version to install some packages:
# python -m pip install pip==9.0.3
# !pip install ./bbp-services-0.0.15/ ./task-types-0.0.10/ 
# !pip install hbp_service_client

In [ ]:
## Testing the average values from a whole POPULATION
models_path = './models_excerpt-pop'
neuroM_stats_file = 'fs_cells_NeuroM_MorphStats_pred.json'

In [ ]:
!pip uninstall --yes morphounit
# !pip install --quiet /home/pedroernesto/Documents/Project/Code/Models_Validation/Validation/ValidationTests/morphounit/
!pip install --upgrade -q git+https://github.com/pedroernesto/morphounit

In [ ]:
## Testing the average values from the whole POPULATION
from morphounit.utils import NeuroM_MorphStats_pop
import morphounit.tests.morph_cells as cells_tests

# To run the test Locally
import os, json, copy
test_obs_path = os.path.expanduser('~/Documents/Project/Code/Models_Validation/Validation/ValidationTests/morphounit_expData')
# obs_file = os.path.join(obs_path, 'fs_morpho_stats_NeuroM.json')
obs_file = os.path.join(test_obs_path, 'fs_morpho_stats_NeuroM_NO_Field_Diam_test.json')
with open(obs_file, 'r+') as fp:
    raw_observation = json.load(fp)

morph_path = models_path
model_name = 'population_'+os.path.basename(morph_path) # Cells-dir
morph_model = NeuroM_MorphStats_pop(model_name=model_name, morph_path=morph_path, 
                                neuroM_pred_file=neuroM_stats_file)

print('Testing population directory ----> ', models_path, '\n\n')

test = cells_tests.NeuroM_MorphStats_pop_Test(observation=copy.deepcopy(raw_observation),
                                              base_directory=morph_model.morph_stats_output)

# test.generate_prediction(model=morph_model)import morphounit.tests.morph_cells as cells_tests

score = test.judge(morph_model, deep_error=True)
print("Score: ", score.score)
print("---------------------------------------------- \n")

if "figures" in score.related_data:
    print("Output files: ")
    for item in score.related_data["figures"]:
        print(item)
print("----------------------------------------------\n")

In [ ]:
# On the Collab
!pip install --upgrade -q hbp_validation_framework

# First login in into the production version of the VF
from hbp_validation_framework import utils, TestLibrary, ModelCatalog
HBP_USERNAME = "pedroernesto"
testLibrary = TestLibrary(username=HBP_USERNAME, environment="production")
# model_catalog = ModelCatalog.from_existing(testLibrary)

# model_catalog=ModelCatalog(username=HBP_USERNAME)

from datetime import datetime
morph_pop_path = models_path
model_pop_name = 'population_'+os.path.basename(morph_path) # Cells-dir
morph_model = NeuroM_MorphStats_pop(model_name=model_pop_name, morph_path=morph_pop_path, 
                                    neuroM_pred_file=neuroM_stats_file)

# Creating a new model instance on the Model Catalog for the chosen 
# cells collection that passed the hard-constraints Test
model_catalog = ModelCatalog(username=HBP_USERNAME)
model_id = '078d19ae-8107-476e-8efe-bf7b0e0898bd'

morph_model.model_uuid = model_id          # uuid for the chosen cell collection
morph_model.model_version = datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+ model_pop_name # model instance

model_catalog.add_model_instance(model_id=morph_model.model_uuid, alias='NeuroM-MorphStats-Pop', 
                                 version=morph_model.model_version)
from time import sleep
sleep(10)

In [ ]:
# Running the test and registering the results on the Collab with the raw functions of the Python client
# Note: observation path is registered with Test definition on the VF!)

import morphounit.tests
test_alias = 'morph_stats_pop_Test'

neuroM_stats_file = 'fs_cells_NeuroM_MorphStats_pred.json'

result_uuids = []

print('Testing population directory ----> ', morph_pop_path, '\n\n')
collab_id="76278"
result_id, score = utils.run_test(username=HBP_USERNAME,
                                  model=morph_model, test_alias=test_alias, test_version="2.0",
                                  storage_collab_id=collab_id, register_result=True, 
                                  client_obj=testLibrary)
result_uuids.append(result_id)

In [ ]:
test_alias="morph_stats_pop_Test"
test_version="1.0"
test = testLibrary.get_validation_test(alias=test_alias, version=test_version)
print(test.description)
test.observation
score = test.judge(morph_model, deep_error=True)

print("----------------------------------------------")
print("Score: ", score.score)
if "figures" in score.related_data:
    print("Output files: ")
    for item in score.related_data["figures"]:
        print(item)
print("----------------------------------------------")

# Registering the results on the VF
from hbp_validation_framework.datastores import CollabDataStore
from datetime import datetime

# collab_id="5165" # Neuroinfo-Gif (dev) Collab
collab_id="8123" # Model validation suites Collab
timestamp = datetime.now().strftime("%Y-%m-%d")
collab_folder = "validation_results/{}/{}_{}".format(timestamp,model_name, datetime.now().strftime("%Y%m%d-%H%M%S"))
collab_storage = CollabDataStore(collab_id=collab_id,
                                 base_folder=collab_folder,
                                 auth=testLibrary.auth)

# HBP authentication token might expire when the test takes too long, 
# so create a new testLibrary object with a valid token
testLibrary = TestLibrary(username=HBP_USERNAME, environment="production")
response = testLibrary.register_result(test_result=score, data_store=collab_storage, project=collab_id)

"""
result_uuids = []
result_id = utils.run_test(hbp_username=HBP_USERNAME, environment="production", 
                           model=morph_model, test_alias=test_alias, test_version=test_version,
                           storage_collab_id=collab_id, register_result=False, client_obj=testLibrary)

result_uuids.append(result_id)
"""

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        valid_uuids, report_path = utils.generate_report(hbp_username=HBP_USERNAME, environment="dev", 
                                                         result_list=result_uuids, only_combined=True)

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))

In [ ]:
# Create a new model entry in the ModelCatalog; metadata to be used for registering model
!pip install --upgrade -q hbp_validation_framework

from hbp_validation_framework import ModelCatalog, TestLibrary
HBP_USERNAME = "pedroernesto"
testLibrary = TestLibrary(username=HBP_USERNAME)
model_catalog = ModelCatalog.from_existing(testLibrary)

model_uuid = model_catalog.register_model(app_id="395557", name="Population of fs_morphologies",
                alias="NeuroM-MorphStats-Pop", author="Alexander Kozlov", organization="KTH-UNIC",
                private=False, cell_type="Fast spiking interneuron", model_type="Other",
                brain_region="Striatum", species="Mouse (Mus musculus)",
                description="This model is being used to demonstrate use of MorphoUnit validation test suite!",
                instances=[{"source":"http://collab.link/8123/usecases/models/basal_morphology/fs",
                            "version":"1.0", "parameters":""}])

In [ ]:
# Register a new test on the test library
name="Basal Ganglia FS-Population Morphology Soft Constraints"
alias="morph_stats_pop_Test"
version="1.0"
author="Pedro Garcia-Rodriguez"
species="Undefined"
age="Any"
brain_region="Striatum"
cell_type="Other"
data_modality="electron microscopy"
test_type="network structure"
score_type="Other"
protocol="Tests a set of average morphological features describing a population of digitally-reconstructed neurons"
data_location="collab://8123/test_observations/morphounit/FSI_morpho_features_NeuroM.json"
data_type="Morphology"
publication="Cutuli (2011), DOI:10.1016/j.nbd.2010.12.007; Ransome (2005), DOI:10.1016/j.neuroscience.2004.12.041; Tepper (2010), DOI:10.3389/fnana.2010.00150; Gittis (2011), DOI:10.1016/j.neuron.2011.06.035; Plenz and Kitai (1998) J Neurosci. 18(1):266-83; Kawaguchi (1990), PMID: 1698947, J Neurosci. 1990 Oct;10(10):3421-38; Fieblinger (2010), DOI: 10.1038/ncomms6316; Preston (1980), PMID: 7353139, Brain Res. 183(2):253-63; Reinius (2015), DOI: 10.3389/fnbeh.2015.00071"
repository="https://github.com/pedroernesto/morphounit.git"
path="morphounit.tests.NeuroM_MorphStats_pop_Test"

from hbp_validation_framework import TestLibrary
HBP_USERNAME = "pedroernesto"
test_library = TestLibrary(username=HBP_USERNAME, environment="dev")

test = test_library.add_test(name=name, alias=alias, version=version, author=author,
                        species=species, age=age, brain_region=brain_region, cell_type=cell_type,
                        data_modality=data_modality, test_type=test_type, score_type=score_type,
                        data_location=data_location, data_type=data_type, publication=publication,
                        repository=repository, path=path)